#Segmenting and Clustering Neighborhoods in Toronto | Part-1
##Scraping Wikipedia page and creating a Dataframe and Transforming the data on Wiki page into pandas dataframe.


Importing Libraries

In [ ]:
# I
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
print("Imported!")

Imported!


Using BeautifulSoup Scraping List of Postal Codes of Given Wikipedia Page


In [44]:
postal_member = []

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res, "html.parser")

# identify table we want to scrape
postal_table = soup.find('table')
print((postal_table))


<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a hr

In [50]:
# loop through table, grab each of the 3 columns shown
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [51]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M4Y,North York,St. James Town
freq,1,24,1


Data Cleaning | Drop None rows of df and row which contains 'Not assigned' value | All "Not assigned" will be replace to 'NaN'


In [53]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [54]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [55]:
df.shape

(103, 3)

## Part 2


Adding Latitude and Longitude Co-ordinates to the DataFrame

In [56]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.8MB/s 


In [58]:
import geocoder

get latitude and longitude function


In [59]:
def get_position(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_position('M4G')

[43.709020000000066, -79.36348999999996]

In [62]:
postal_codes = df['PostalCode'] 
position = [ get_position(postal_code) for postal_code in postal_codes.tolist() ]
df_coords = pd.DataFrame(position, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [63]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


##Part 3

In [64]:
df['Borough'].unique()


array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East York/East Toronto', 'Central Toronto', 'Mississauga',
       'Downtown Toronto Stn A', 'Etobicoke Northwest',
       'East Toronto Business'], dtype=object)

In [66]:
df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop = True)
df_toronto.shape

(39, 5)

There are 39 neighborhoods in Toronto

import libraries

In [67]:
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import webbrowser
from IPython.display import Image

Get the coordinate of Toronto

In [68]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Plot them on the map

In [69]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

Use the Foursquare API to explore the neighborhoods and segment them

In [70]:
CLIENT_ID = 'CURLH5YYCXMLJUABNE5Y22LK1JNKWHZLO5MCW2OD4PRRRDK1' # your Foursquare ID
CLIENT_SECRET = 'O5PCL405KIK4MGGBIMJD2EIAYSEIQK03W4QMEG4L4ZYOEMMF' # your Foursquare Secret
VERSION = '20210425' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CURLH5YYCXMLJUABNE5Y22LK1JNKWHZLO5MCW2OD4PRRRDK1
CLIENT_SECRET:O5PCL405KIK4MGGBIMJD2EIAYSEIQK03W4QMEG4L4ZYOEMMF


Create a function to explore all neighborhoods in Toronto


In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
LIMIT = 20 # limit of number of venues returned by Foursquare API

toronto_venues = getNearbyVenues(names = df_toronto['Neighborhood'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

check the size of the resulting dataframe


In [74]:
print(toronto_venues.shape)
toronto_venues.head()

(571, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


check the number of venues returned for each neighborhood


In [75]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton, Parkdale Village, Exhibition Place",20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
Christie,10,10,10,10,10,10
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20
Davisville North,8,8,8,8,8,8


In [76]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 158 uniques categories.


In [77]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(571, 158)


,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,...,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and take mean of the frequency of occurrence of each category


In [78]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [79]:
print(toronto_grouped.shape)
toronto_grouped

(39, 158)


,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,...,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.050000,0.000000,0.000000,0.00,0.10,0.00,0.00,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.050000,0.0,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.00,...,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.000000,0.00,0.0,0.100000,0.000000,0.050000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.050,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.05,0.100000,0.00,0.00,...,0.00,0.05,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.100000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.0,0.050000,0.000000,0.000000,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.050000,0.0,0.05,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,...,0.05,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.05,0.00,0.050000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.000000,0.05,0.05,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
3,Central Bay Street,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000,0.00,0.05,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.00,0.05,0.05,0.00,0.150000,0.00,0.00,...,0.00,0.00,0.000000,0.05,0.000000,0.05,0.05,0.00,0.05,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.00,0.00,0.00,0.000000,0.05,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
4,Christie,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.1,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.300000,0.1,0.00,0.00,0.00,0.00,0.00,0.100000,0.00,0.00,...,0.00,0.00,0.000000,0.00,0.100000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
5,Church and Wellesley,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.000000,0.050,0.00,0.05,0.05,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.100000,0.00,0.00,...,0.00,0.00,0.000000,0.05,0.000000,0.00,0.00,0.05,0.00,0.00,0.050000,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.05,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0

sort the venues in descending order

In [80]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe with the top 10 venues for each neighborhood


In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Beer Bar,Cocktail Bar,Food Truck,Museum,Liquor Store,Café,Fish Market,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Restaurant,Cocktail Bar,Pet Store,Japanese Restaurant,Italian Restaurant,Sandwich Place,Spa,Hotel
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Park,Peruvian Restaurant,Sandwich Place,Ramen Restaurant,Hotel,Spa,Speakeasy,Café,Beer Bar
3,Central Bay Street,Coffee Shop,Clothing Store,Sushi Restaurant,Electronics Store,Comic Shop,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Chinese Restaurant,Pizza Place
4,Christie,Café,Grocery Store,Candy Store,Coffee Shop,Italian Restaurant,Playground,Baby Store,Wine Shop,Eastern European Restaurant,Donut Shop


K-means clustering neighborhoods


In [82]:
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 5, 0, 5, 6, 5, 6, 0, 0, 0], dtype=int32)

Add clustering labels


In [83]:
neighborhoods_venues_cluster = neighborhoods_venues_sorted.copy()
neighborhoods_venues_cluster.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Beer Bar,Cocktail Bar,Food Truck,Museum,Liquor Store,Café,Fish Market,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Restaurant,Cocktail Bar,Pet Store,Japanese Restaurant,Italian Restaurant,Sandwich Place,Spa,Hotel
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Park,Peruvian Restaurant,Sandwich Place,Ramen Restaurant,Hotel,Spa,Speakeasy,Café,Beer Bar
3,Central Bay Street,Coffee Shop,Clothing Store,Sushi Restaurant,Electronics Store,Comic Shop,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Chinese Restaurant,Pizza Place
4,Christie,Café,Grocery Store,Candy Store,Coffee Shop,Italian Restaurant,Playground,Baby Store,Wine Shop,Eastern European Restaurant,Donut Shop


In [84]:
neighborhoods_venues_cluster.insert(0, 'Cluster Labels', kmeans.labels_)


In [85]:
toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_cluster.set_index('Neighborhood'), on = 'Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,5,Coffee Shop,Breakfast Spot,Yoga Studio,Food Truck,Event Space,Italian Restaurant,Restaurant,Distribution Center,Spa,Pub
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,6,Café,Clothing Store,Hotel,Music Venue,Pizza Place,Plaza,Diner,Ramen Restaurant,Burrito Place,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,6,Gastropub,Coffee Shop,Food Truck,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Café,Restaurant,Japanese Restaurant,Hotel
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,9,Trail,Health Food Store,Pub,Wine Shop,Deli / Bodega,Eastern European Restaurant,Donut Shop,Distribution Center,Discount Store,Diner
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,0,Seafood Restaurant,Farmers Market,Beer Bar,Cocktail Bar,Food Truck,Museum,Liquor Store,Café,Fish Market,Bakery


Create a map

In [86]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters